In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

In [10]:
v_market = 2;
k = 120
tau = 1;
r = 0.05
s_0 = 100;
sigmaInit = 0.10
CP="C"

def ImpliedVolatility(CP,s_0,k,sigma,tau,r):
    error = 1e10;
    optionPrice = lambda sigma:BS_Call_Option(CP,s_0,k,sigma,tau,r)
    vega = lambda sigma:dV_dsigma(s_0,k,sigma,tau,r)
    
    n = 1.0
    while error>10e-10:
        f = v_market-optionPrice(sigma)
        f_prim = -vega(sigma)
        sigma_new = sigma-f/f_prim
        
        error = abs(sigma_new-sigma);
        sigma = sigma_new
        
    print('iteration {0} with error = {1}'.format(n,error))
    n = n+1
    return sigma

#vega calculation

def dV_dsigma(s_0,k,sigma,tau,r):
    d2 = (np.log(s_0/float(k))+(r-0.5*sigma**2)*tau)/float(sigma*(tau**0.5))
    value = k*np.exp(-r*tau)*st.norm.cdf(d2)*np.sqrt(tau)
    return value

def BS_Call_Option(CP,s_0,k,sigma,tau,r):
    d1 = (np.log(s_0/float(k))+(r+0.5*sigma**2)*tau)/float(sigma*(tau**0.5))
    d2 = d1-sigma*np.sqrt(tau)
    
    if str(CP).lower() == "c" or str(CP).lower() == "1":
        value = s_0*st.norm.cdf(d1) - k*np.exp(-r*tau)*st.norm.cdf(d2)
    elif str(CP).lower() == "p" or str(CP).lower() == "-1":
        value = -(s_0*st.norm.cdf(d1)) + k*np.exp(-r*tau)*st.norm.cdf(d2)
    return value


sigma_imp = ImpliedVolatility(CP,s_0,k,sigmaInit,tau,r)
message = '''Implied volatility for CallPrice= {}, strike K={}, 
      maturity T= {}, interest rate r= {} and initial stock S_0={} 
      equals to sigma_imp = {:.7f}'''.format(v_market,k,tau,r,s_0,sigma_imp)
            
print(message)

# Check! 
val = BS_Call_Option(CP,s_0,k,sigma_imp,tau,r)
print('Option Price for implied volatility of {0} is equal to {1}'.format(sigma_imp, val))

iteration 1.0 with error = 4.873423609108585e-10
Implied volatility for CallPrice= 2, strike K=120, 
      maturity T= 1, interest rate r= 0.05 and initial stock S_0=100 
      equals to sigma_imp = 0.1614827
Option Price for implied volatility of 0.16148272899165422 is equal to 2.000000004563198
